In [34]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import itertools
import pandas as pd
from datetime import datetime

In [ ]:
#load graph
sample=pd.read_csv("Amazon_rating_sample4.csv")
sample.drop(['Timestamp'], axis = 1, inplace = True)
sample.drop(['Unnamed: 0'], axis = 1, inplace = True)
sample

In [191]:
sample=pd.read_csv("rec-amazon-ratings.edges",header=None,delimiter=" ")
sample

,0,1,2,3
0,1,1,5.0,1117404000
1,1,2,1.0,1105916400
2,1,3,5.0,1105916400
3,1,4,1.0,1105570800
4,1,5,1.0,1104966000
...,...,...,...,...
5838036,2146054,1230915,5.0,1135983600
5838037,2146054,174742,3.0,1122328800
5838038,2146055,140015,2.0,1104447600
5838039,2146056,592972,5.0,929743200


In [211]:
a=sample[sample[2]==5]
b=sample[sample[2]==4]
sample2=pd.concat([a,b],axis=0)

In [212]:
sample2

,0,1,2,3
0,1,1,5.0,1117404000
2,1,3,5.0,1105916400
5,1,6,5.0,1103497200
7,1,8,5.0,1074985200
8,1,9,5.0,1071961200
...,...,...,...,...
5838026,2146048,2874,4.0,1124488800
5838029,2146050,765079,4.0,1128981600
5838030,2146050,8732,4.0,1125525600
5838034,2146052,233344,4.0,1125871200


In [218]:
G = nx.Graph()
# Add edges and edge attributes
temp=0
for i, elrow in sample2.iterrows():
    if (temp%100000==0):
        print(temp/100000,"/45-->",sep="")
    temp=temp+1
    G.add_edge(elrow[0], elrow[1])

0.0/45-->
1.0/45-->
2.0/45-->
3.0/45-->
4.0/45-->
5.0/45-->
6.0/45-->
7.0/45-->
8.0/45-->
9.0/45-->
10.0/45-->
11.0/45-->
12.0/45-->
13.0/45-->
14.0/45-->
15.0/45-->
16.0/45-->
17.0/45-->
18.0/45-->
19.0/45-->
20.0/45-->
21.0/45-->
22.0/45-->
23.0/45-->
24.0/45-->
25.0/45-->
26.0/45-->
27.0/45-->
28.0/45-->
29.0/45-->
30.0/45-->
31.0/45-->
32.0/45-->
33.0/45-->
34.0/45-->
35.0/45-->
36.0/45-->
37.0/45-->
38.0/45-->
39.0/45-->
40.0/45-->
41.0/45-->
42.0/45-->
43.0/45-->
44.0/45-->
45.0/45-->


In [219]:
#G = nx.read_edgelist("rec-amazon-ratings.edges",  create_using=nx.DiGraph,nodetype= int).to_undirected()
print("full_data except rating under 4")
print ("number of nodes :",len(G.nodes))  
print ("number of edges :",len(G.edges))  

full_data except rating under 4
number of nodes : 1937490
number of edges : 4454690


In [226]:
G1=G.copy()
for i in list(G1.nodes):
    check=np.random.randint(10)
    if not check==2:
        G1.remove_node(i)

print("%10 sample of full_data")
print ("number of nodes :",len(G1.nodes))  
print ("number of edges :",len(G1.edges))  

%10 sample of full_data
number of nodes : 193742
number of edges : 46029


In [228]:
G2 = max(nx.connected_component_subgraphs(G1), key=len)
print("giant component of %10 sample of full_data")
print ("number of nodes :",len(G2.nodes))  
print ("number of edges :",len(G2.edges))  

giant component of %10 sample of full_data
number of nodes : 27510
number of edges : 32791


In [229]:
deg = dict(nx.degree(G2))
sorteddeg = sorted(deg.items(), key=lambda x: x[1], reverse=False)
small_deg=[]
for k, v in sorteddeg:
    if v<3:
        G2.remove_node(k)
print("only degree greater than 3 of giant component of %50 sample of full_data")
print ("number of nodes :",len(G2.nodes))  
print ("number of edges :",len(G2.edges))  

only degree greater than 3 of giant component of %50 sample of full_data
number of nodes : 5612
number of edges : 9040


In [230]:
G3 = max(nx.connected_component_subgraphs(G2), key=len)
print("giant component of only degree greater than 2 of giant component of %50 sample of full_data")
print ("number of nodes :",len(G3.nodes))  
print ("number of edges :",len(G3.edges))  

giant component of only degree greater than 2 of giant component of %50 sample of full_data
number of nodes : 5225
number of edges : 8972


# NO NEED TO DO THE SAMPLING OPERATIONS

In [35]:
G3= nx.read_edgelist("Full.edgelist", create_using= nx.DiGraph, nodetype= float).to_undirected()
print("Full data")
print ("number of nodes :",len(G3.nodes))  
print ("number of edges :",len(G3.edges)) 

Full data
number of nodes : 5225
number of edges : 8972


In [36]:
total=len(G3.nodes)

In [37]:
def getTrainTestMatrix (Graph,splitsize):
    allEdges = []
    testEdges = []
    trainEdges = []
    
    for edge in Graph.edges():
        pair = edge
        # Ensure that tuple (u,v) u < v
        if pair[0] > pair[1]:
            allEdges.append((pair[1], pair[0]))
        else:
            allEdges.append((pair[0], pair[1]))

    numTest = int(len(allEdges) * splitsize)

    # sample 
    testEdgesInd = np.random.choice(range(len(allEdges)), numTest, replace=False)
    for ind in range(len(allEdges)):
        pair = allEdges[ind]
        if ind in testEdgesInd:
            testEdges.append(pair)
        else:
            trainEdges.append(pair)

    numTrain = len(trainEdges)
    assert numTest==len(testEdges)

    # start filling in train and test matrix
    XtrainPos = np.zeros(shape=(numTrain,5))
    XtrainNeg = np.zeros(shape=(numTrain,5))
    YtrainPos = np.ones(shape=(numTrain,1))
    YtrainNeg = np.zeros(shape=(numTrain,1))

    Xtest = np.zeros(shape=(100000000,5))
    Ytest = np.zeros(shape=(100000000,1))

    trainPosCount = 0
    trainNegCount = 0
    testCount = 0
###############################BURADA KALDIM
    total=len(G3.nodes)
    temp=1
    now = datetime.now()
    current_timea=now 
    for node1 in Graph.nodes():
        for node2 in Graph.nodes():
            node1Id = node1
            node2Id = node2
            if node2Id <= node1Id:
                continue
           
            deg1 = Graph.degree[node1]
            deg2 = Graph.degree[node2]
            
            preds = nx.preferential_attachment(Graph, [(node1,node2)])  
            u,v,p=next(preds)
            pa = p
            
            
            preds = nx.jaccard_coefficient(Graph, [(node1,node2)])  
            u,v,p=next(preds)
            ja =p
            preds= nx.adamic_adar_index(Graph, [(node1,node2)]) 
            u,v,p=next(preds)
            aa = p
        
            if (node1Id, node2Id) in trainEdges:
                XtrainPos[trainPosCount][0] = deg1
                XtrainPos[trainPosCount][1] = deg2
                XtrainPos[trainPosCount][2] = pa
                XtrainPos[trainPosCount][3] = ja
                XtrainPos[trainPosCount][4] = aa
                trainPosCount += 1
            
            elif (node1Id, node2Id) in testEdges:
                Xtest[testCount][0] = deg1
                Xtest[testCount][1] = deg2
                Xtest[testCount][2] = pa
                Xtest[testCount][3] = ja
                Xtest[testCount][4] = aa
                Ytest[testCount][0] = 1
                testCount += 1

            else: # this is not an edge

                
                
                if trainNegCount < 1500:
                    if (trainNegCount%100==0):
                        print ("train neg: "+str(trainNegCount))
                    XtrainNeg[trainNegCount][0] = deg1
                    XtrainNeg[trainNegCount][1] = deg2
                    XtrainNeg[trainNegCount][2] = pa
                    XtrainNeg[trainNegCount][3] = ja
                    XtrainNeg[trainPosCount][4] = aa
                    trainNegCount += 1
                else:
                    Xtest[testCount][0] = deg1
                    Xtest[testCount][1] = deg2
                    Xtest[testCount][2] = pa
                    Xtest[testCount][3] = ja
                    Xtest[testCount][4] = aa
                    Ytest[testCount][0] = 0
                    testCount += 1
        if (temp%10==0):
            print("nodes completed :",(temp/total))
            now = datetime.now()
            current_timeb=now
            print("difference =", (current_timeb-current_timea))
        temp=temp+1
        now = datetime.now()
        current_timea=now
    #assert trainPosCount==numTrain
    #assert trainNegCount==1500

    testXFull = Xtest[0:testCount, :]
    testYFull = Ytest[0:testCount, :]
    trainXFull = np.concatenate((XtrainPos, XtrainNeg))
    trainYFull = np.concatenate((YtrainPos, YtrainNeg))

    return (trainXFull, trainYFull, testXFull, testYFull)


In [38]:
(Xtrain, Ytrain, Xtest, Ytest) = getTrainTestMatrix(G3,0.2)

train neg: 0
train neg: 100
train neg: 200
train neg: 300
train neg: 400
train neg: 500
train neg: 600
train neg: 700
train neg: 800
train neg: 900
train neg: 1000
train neg: 1100
train neg: 1200
train neg: 1300
train neg: 1400
nodes completed : 0.0019138755980861245
difference = 0:00:01.805177
nodes completed : 0.003827751196172249
difference = 0:00:02.340737
nodes completed : 0.005741626794258373
difference = 0:00:02.307831
nodes completed : 0.007655502392344498
difference = 0:00:02.189148
nodes completed : 0.009569377990430622
difference = 0:00:01.397264
nodes completed : 0.011483253588516746
difference = 0:00:02.348690
nodes completed : 0.013397129186602871
difference = 0:00:01.677050
nodes completed : 0.015311004784688996
difference = 0:00:00.753986
nodes completed : 0.01722488038277512
difference = 0:00:01.895931
nodes completed : 0.019138755980861243
difference = 0:00:01.998622
nodes completed : 0.021052631578947368
difference = 0:00:00.936532
nodes completed : 0.022966507177033

nodes completed : 0.23349282296650717
difference = 0:00:00.877657
nodes completed : 0.23540669856459331
difference = 0:00:01.082105
nodes completed : 0.23732057416267943
difference = 0:00:00.590426
nodes completed : 0.23923444976076555
difference = 0:00:00.674210
nodes completed : 0.24114832535885167
difference = 0:00:00.827784
nodes completed : 0.2430622009569378
difference = 0:00:00.847736
nodes completed : 0.24497607655502393
difference = 0:00:01.532904
nodes completed : 0.24688995215311005
difference = 0:00:02.135292
nodes completed : 0.24880382775119617
difference = 0:00:01.597540
nodes completed : 0.2507177033492823
difference = 0:00:01.645572
nodes completed : 0.25263157894736843
difference = 0:00:02.055504
nodes completed : 0.2545454545454545
difference = 0:00:01.790209
nodes completed : 0.25645933014354066
difference = 0:00:01.885955
nodes completed : 0.2583732057416268
difference = 0:00:01.721410
nodes completed : 0.2602870813397129
difference = 0:00:00.615392
nodes completed

nodes completed : 0.47464114832535886
difference = 0:00:00.875661
nodes completed : 0.47655502392344495
difference = 0:00:00.228349
nodes completed : 0.4784688995215311
difference = 0:00:01.512956
nodes completed : 0.48038277511961724
difference = 0:00:01.224726
nodes completed : 0.48229665071770333
difference = 0:00:00.415890
nodes completed : 0.4842105263157895
difference = 0:00:00.185464
nodes completed : 0.4861244019138756
difference = 0:00:01.258635
nodes completed : 0.4880382775119617
difference = 0:00:01.318475
nodes completed : 0.48995215311004786
difference = 0:00:01.804178
nodes completed : 0.49186602870813395
difference = 0:00:01.345403
nodes completed : 0.4937799043062201
difference = 0:00:00.786895
nodes completed : 0.49569377990430624
difference = 0:00:00.345078
nodes completed : 0.49760765550239233
difference = 0:00:01.076156
nodes completed : 0.4995215311004785
difference = 0:00:00.718079
nodes completed : 0.5014354066985646
difference = 0:00:00.856674
nodes completed :

nodes completed : 0.7177033492822966
difference = 0:00:00.873670
nodes completed : 0.7196172248803828
difference = 0:00:00.159574
nodes completed : 0.7215311004784689
difference = 0:00:01.297496
nodes completed : 0.723444976076555
difference = 0:00:01.332440
nodes completed : 0.7253588516746412
difference = 0:00:01.024263
nodes completed : 0.7272727272727273
difference = 0:00:00.272309
nodes completed : 0.7291866028708134
difference = 0:00:00.730090
nodes completed : 0.7311004784688995
difference = 0:00:01.333398
nodes completed : 0.7330143540669857
difference = 0:00:00.500660
nodes completed : 0.7349282296650718
difference = 0:00:01.208801
nodes completed : 0.7368421052631579
difference = 0:00:00.397966
nodes completed : 0.738755980861244
difference = 0:00:00.666216
nodes completed : 0.7406698564593301
difference = 0:00:00.670208
nodes completed : 0.7425837320574162
difference = 0:00:01.320451
nodes completed : 0.7444976076555024
difference = 0:00:01.328449
nodes completed : 0.7464114

nodes completed : 0.9607655502392345
difference = 0:00:01.178850
nodes completed : 0.9626794258373206
difference = 0:00:00.078826
nodes completed : 0.9645933014354067
difference = 0:00:00.825829
nodes completed : 0.9665071770334929
difference = 0:00:01.149926
nodes completed : 0.968421052631579
difference = 0:00:00.045876
nodes completed : 0.970334928229665
difference = 0:00:00.793878
nodes completed : 0.9722488038277513
difference = 0:00:00.833769
nodes completed : 0.9741626794258373
difference = 0:00:00.761926
nodes completed : 0.9760765550239234
difference = 0:00:00.225396
nodes completed : 0.9779904306220095
difference = 0:00:00.449797
nodes completed : 0.9799043062200957
difference = 0:00:00.281248
nodes completed : 0.9818181818181818
difference = 0:00:00.742017
nodes completed : 0.9837320574162679
difference = 0:00:00.079783
nodes completed : 0.9856459330143541
difference = 0:00:00.508681
nodes completed : 0.9875598086124402
difference = 0:00:00.617313
nodes completed : 0.9894736

In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [40]:
#create pipeline for a machine learning model-random forest classifier, grid search, and cv=3, report evaluation metrics
pipe1 = Pipeline([
                 ("scaler", StandardScaler()),
                 ("rnd", RandomForestClassifier())
                 ])

#pipe.fit(X_train, y_train)
#y_pred = pipe.predict(X_test)

param_grid1 = {
    "rnd__max_depth": [2, 3, 5],
    "rnd__n_estimators": [5, 10, 15]
}
from sklearn.model_selection import  GridSearchCV

grid_search1 = GridSearchCV(pipe1, param_grid1, cv=3)
grid_search1.fit(Xtrain,Ytrain)
 
    
print(("best regression from grid search: %.3f"
           % grid_search1.score(Xtest, Ytest)))
print(grid_search1.best_params_)
y_pred=grid_search1.best_estimator_.predict(Xtest)

from sklearn.metrics import f1_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
print("model1")
print("Random Forest Classifier")
print('r2 score: '+str(r2_score(Ytest, y_pred)))
print('RMSE : '+str(np.sqrt(mean_squared_error(Ytest, y_pred))))
print("f1 score: "+str(f1_score(Ytest, y_pred, average='macro')))
print("accuracy score:", accuracy_score(Ytest, y_pred))
print("recall score:", recall_score(Ytest, y_pred))
print("precision score:", precision_score(Ytest, y_pred))

C:\Users\sinan.turkmen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\sinan.turkmen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\sinan.turkmen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\pipeline.py:354: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\sinan.turkmen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\pipel

best regression from grid search: 0.000
{'rnd__max_depth': 2, 'rnd__n_estimators': 5}
model1
Random Forest Classifier
r2 score: -7601.577480490474
RMSE : 0.9999342306679337
f1 score: 0.00013151703952065042
accuracy score: 0.00013153433852744888
recall score: 1.0
precision score: 0.00013153433852744888


In [41]:
#create pipeline for a machine learning model-Stochastic Gradient Descent  classifier, grid search, and cv=3, report evaluation metrics
pipe2 = Pipeline([
                 ("scaler", StandardScaler()),
                 ("sgd", SGDClassifier())
                 ])

#pipe.fit(X_train, y_train)
#y_pred = pipe.predict(X_test)

param_grid2 = {
    "sgd__loss": ["huber",],
    "sgd__penalty": ["l2", "l1", "elasticnet"]
}
from sklearn.model_selection import  GridSearchCV

grid_search2 = GridSearchCV(pipe2, param_grid2, cv=3)
grid_search2.fit(Xtrain,Ytrain)
 
    
print(("best regression from grid search: %.3f"
           % grid_search2.score(Xtest, Ytest)))
print(grid_search2.best_params_)
y_pred=grid_search2.best_estimator_.predict(Xtest)



from sklearn.metrics import f1_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
print("model2")
print("Stochastic Gradient Descent Classifier")
print('r2 score: '+str(r2_score(Ytest, y_pred)))
print('RMSE : '+str(np.sqrt(mean_squared_error(Ytest, y_pred))))
print("f1 score: "+str(f1_score(Ytest, y_pred, average='macro')))
print("accuracy score:", accuracy_score(Ytest, y_pred))
print("recall score:", recall_score(Ytest, y_pred))
print("precision score:", precision_score(Ytest, y_pred))

C:\Users\sinan.turkmen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sinan.turkmen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sinan.turkmen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sinan.turkmen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: D

best regression from grid search: 0.975
{'sgd__loss': 'huber', 'sgd__penalty': 'l1'}
model2
Stochastic Gradient Descent Classifier
r2 score: -190.2693056387872
RMSE : 0.15860382212786414
f1 score: 0.49491586586952013
accuracy score: 0.9748448276064329
recall score: 0.2463768115942029
precision score: 0.001291710259452572


In [42]:
#create pipeline for a machine learning model-Stochastic Gradient Descent  classifier, grid search, and cv=3, report evaluation metrics
pipe3 = Pipeline([
                 ("scaler", StandardScaler()),
                 ("dec", DecisionTreeClassifier(random_state=0))
                 ])

#pipe.fit(X_train, y_train)
#y_pred = pipe.predict(X_test)

param_grid3 = {
    #"dec__loss": ["huber",],
    "dec__criterion": ["gini", "entropy"]
}
from sklearn.model_selection import  GridSearchCV

grid_search3 = GridSearchCV(pipe3, param_grid3, cv=3)
grid_search3.fit(Xtrain,Ytrain)
 
    
print(("best regression from grid search: %.3f"
           % grid_search3.score(Xtest, Ytest)))
print(grid_search3.best_params_)
y_pred=grid_search3.best_estimator_.predict(Xtest)



from sklearn.metrics import f1_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
print("model3")
print("Decision Tree Classifier")
print('r2 score: '+str(r2_score(Ytest, y_pred)))
print('RMSE : '+str(np.sqrt(mean_squared_error(Ytest, y_pred))))
print("f1 score: "+str(f1_score(Ytest, y_pred, average='macro')))
print("accuracy score:", accuracy_score(Ytest, y_pred))
print("recall score:", recall_score(Ytest, y_pred))
print("precision score:", precision_score(Ytest, y_pred))

best regression from grid search: 0.105
{'dec__criterion': 'gini'}
model3
Decision Tree Classifier
r2 score: -6803.135780174637
RMSE : 0.9459702843233944
f1 score: 0.09518464859808005
accuracy score: 0.10514022117711648
recall score: 0.9336677814938684
precision score: 0.00013722118498480737


In [43]:
import xgboost
#create pipeline for a machine learning model-Stochastic Gradient Descent  classifier, grid search, and cv=3, report evaluation metrics
pipe4 = Pipeline([
                 ("scaler", StandardScaler()),
                 ("xgb", xgboost.XGBClassifier())
                 ])

#pipe.fit(X_train, y_train)
#y_pred = pipe.predict(X_test)

param_grid4 = {
    #"dec__loss": ["huber",],
    #"dec__criterion": ["gini", "entropy"]
}
from sklearn.model_selection import  GridSearchCV

grid_search4 = GridSearchCV(pipe4, param_grid4, cv=3)
grid_search4.fit(Xtrain,Ytrain)
 
    
print(("best regression from grid search: %.3f"
           % grid_search4.score(Xtest, Ytest)))
print(grid_search4.best_params_)
y_pred=grid_search4.best_estimator_.predict(Xtest)



from sklearn.metrics import f1_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
print("model4")
print("XGBoost Classifier")
print('r2 score: '+str(r2_score(Ytest, y_pred)))
print('RMSE : '+str(np.sqrt(mean_squared_error(Ytest, y_pred))))
print("f1 score: "+str(f1_score(Ytest, y_pred, average='macro')))
print("accuracy score:", accuracy_score(Ytest, y_pred))
print("recall score:", recall_score(Ytest, y_pred))
print("precision score:", precision_score(Ytest, y_pred))

C:\Users\sinan.turkmen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sinan.turkmen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sinan.turkmen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\sinan.turkmen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\

best regression from grid search: 0.044
{}
model4
XGBoost Classifier
r2 score: -7270.802088110067
RMSE : 0.9779396024612003
f1 score: 0.04183030669322183
accuracy score: 0.04363413393802958
recall score: 0.9331103678929766
precision score: 0.00012832059236587528
